## 뉴스 웹 크롤링 

### 크롤링 대상
1. [네이버 뉴스 경제/금융 카테고리 페이지](https://news.naver.com/breakingnews/section/101/259)
2. [네이버 뉴스 경제/증권 카테고리 페이지](https://news.naver.com/breakingnews/section/101/258)
3. [네이버 뉴스 경제/산업,제계 카테고리 페이지](https://news.naver.com/breakingnews/section/101/261)
4. [네이버 뉴스 경제/중기,벤처 카테고리 페이지](https://news.naver.com/breakingnews/section/101/771)
5. [네이버 뉴스 경제/부동산 카테고리 페이지](https://news.naver.com/breakingnews/section/101/260)
6. [네이버 뉴스 경제/글로벌 경제 카테고리 페이지](https://news.naver.com/breakingnews/section/101/262)
7. [네이버 뉴스 경제/생활 경제 카테고리 페이지](https://news.naver.com/breakingnews/section/101/310)
8. [네이버 뉴스 경제/경제 일반 카테고리 페이지](https://news.naver.com/breakingnews/section/101/263)

현재 코드는 특정 URL에서 단일 페이지의 데이터를 가져오며, 한 페이지에서 최대 36개의 뉴스를 수집합니다.

In [18]:
urls = ["https://news.naver.com/breakingnews/section/101/259",
        "https://news.naver.com/breakingnews/section/101/258",
        "https://news.naver.com/breakingnews/section/101/261",
        "https://news.naver.com/breakingnews/section/101/771",
        "https://news.naver.com/breakingnews/section/101/260",
        "https://news.naver.com/breakingnews/section/101/262",
        "https://news.naver.com/breakingnews/section/101/310",
        "https://news.naver.com/breakingnews/section/101/263"
    ]

In [19]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By

# CSV 파일 작성 함수
def save_to_csv(file_name, data, fields):
    """
    데이터를 CSV 파일에 저장하는 함수
    :param file_name: 저장할 CSV 파일 이름
    :param data: 저장할 데이터 (리스트 형식)
    :param fields: CSV 헤더 (리스트 형식)
    """
    with open(file_name, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # 헤더 작성
        writer.writerow(fields)
        # 데이터 작성
        writer.writerows(data)

### 총 8개의 페이지에서 각 페이지당 36개의 기사를 수집하여, 모든 내용을 ‘news_data.csv’ 파일에 저장합니다.

In [21]:
# Selenium을 사용한 웹 크롤링

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time


# WebDriver 초기화 및 페이지 열기
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 데이터 저장 리스트
articles_data = []
fields = ["기사 링크", "기사 제목", "이미지 URL", "기사 요약", "언론사", "게시 시간"]

# 모든 기사 요소 찾기
articles = driver.find_elements(By.CLASS_NAME, "sa_item")


# 각 URL에 대해 크롤링 수행
for url in urls:
    try:
        # 페이지 열기
        driver.get(url)

        # JavaScript 로드 대기
        time.sleep(2)

        # 모든 기사 요소 찾기
        articles = driver.find_elements(By.CLASS_NAME, "sa_item")

        # 각 기사 정보 추출
        for article in articles:
            try:
                # 데이터 추출
                link = article.find_element(By.CSS_SELECTOR, "a.sa_text_title").get_attribute("href")
                title = article.find_element(By.CSS_SELECTOR, "strong.sa_text_strong").text

                # 이미지 URL 추출 (이미지가 없는 경우 처리)
                try:
                    image_url = article.find_element(By.CSS_SELECTOR, "div.sa_thumb img").get_attribute("src")
                except Exception:
                    image_url = None  # 이미지가 없는 경우 기본값 설정

                summary = article.find_element(By.CLASS_NAME, "sa_text_lede").text
                press = article.find_element(By.CLASS_NAME, "sa_text_press").text
                datetime = article.find_element(By.CLASS_NAME, "sa_text_datetime").text

                # 데이터 저장 (리스트 형태)
                articles_data.append([link, title, image_url, summary, press, datetime])

            except Exception as e:
                print(f"기사 처리 중 오류 발생: {e}")
                continue
        # 현재 URL 크롤링 완료 메시지 출력
        print(f"URL 크롤링 완료: {url}")

    except Exception as e:
        print(f"URL 처리 중 오류 발생 ({url}): {e}")
        continue

# 브라우저 닫기
driver.quit()

# CSV 파일로 저장
output_file = "news_data.csv"
save_to_csv(output_file, articles_data, fields)

print(f"CSV 파일 '{output_file}'에 저장 완료!")


URL 크롤링 완료: https://news.naver.com/breakingnews/section/101/259
URL 크롤링 완료: https://news.naver.com/breakingnews/section/101/258
URL 크롤링 완료: https://news.naver.com/breakingnews/section/101/261
URL 크롤링 완료: https://news.naver.com/breakingnews/section/101/771
URL 크롤링 완료: https://news.naver.com/breakingnews/section/101/260
URL 크롤링 완료: https://news.naver.com/breakingnews/section/101/262
URL 크롤링 완료: https://news.naver.com/breakingnews/section/101/310
URL 크롤링 완료: https://news.naver.com/breakingnews/section/101/263
CSV 파일 'news_data.csv'에 저장 완료!


In [30]:
import csv

def extract_links_from_csv(file_name, link_column_name):
    """
    CSV 파일에서 특정 열(기사 링크)에 해당하는 데이터를 리스트로 추출
    :param file_name: CSV 파일 이름
    :param link_column_name: 링크가 저장된 열 이름
    :return: 링크 리스트
    """
    links = []
    
    # CSV 파일 열기
    with open(file_name, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        
        # 각 행에서 링크 추출
        for row in reader:
            if link_column_name in row:
                links.append(row[link_column_name])
    
    return links

# CSV 파일에서 기사 링크 추출
file_name = "news_data.csv"  # 기존 저장된 CSV 파일
link_column_name = "기사 링크"  # 링크가 저장된 열 이름
links = extract_links_from_csv(file_name, link_column_name)

# 결과 확인
print(f"추출된 기사 링크 ({len(links)}개):")
print(links)

추출된 기사 링크 (288개):
['https://n.news.naver.com/mnews/article/011/0004441569', 'https://n.news.naver.com/mnews/article/018/0005927900', 'https://n.news.naver.com/mnews/article/018/0005927897', 'https://n.news.naver.com/mnews/article/417/0001053018', 'https://n.news.naver.com/mnews/article/009/0005431277', 'https://n.news.naver.com/mnews/article/009/0005431276', 'https://n.news.naver.com/mnews/article/009/0005431275', 'https://n.news.naver.com/mnews/article/009/0005431274', 'https://n.news.naver.com/mnews/article/009/0005431273', 'https://n.news.naver.com/mnews/article/009/0005431272', 'https://n.news.naver.com/mnews/article/417/0001053016', 'https://n.news.naver.com/mnews/article/003/0013024054', 'https://n.news.naver.com/mnews/article/014/0005297673', 'https://n.news.naver.com/mnews/article/028/0002727365', 'https://n.news.naver.com/mnews/article/421/0008030807', 'https://n.news.naver.com/mnews/article/366/0001048268', 'https://n.news.naver.com/mnews/article/421/0008030749', 'https://n.n

In [40]:
import random

# 테스트를 위해 랜덤으로 100개의 링크 추출
test_links = random.sample(links, min(100, len(links)))  # 링크가 100개 미만이면 전체 사용

In [25]:
# CSS 선택자를 사용해 요소의 텍스트 가져오기
def fetch_element_text(driver, css_selector, error_message):
    try:
        return driver.find_element(By.CSS_SELECTOR, css_selector).text.strip()
    except Exception as e:
        print(f"{error_message}: {e}")
        return None

# CSS 선택자를 사용해 요소의 속성 값 가져오기
def fetch_element_attribute(driver, css_selector, attribute, error_message):
    try:
        return driver.find_element(By.CSS_SELECTOR, css_selector).get_attribute(attribute)
    except Exception as e:
        print(f"{error_message}: {e}")
        return None


### news_data.csv에 저장된 기사 링크를 이용해 각 링크의 사이트에 접속하고, 해당 페이지에서 뉴스의 세부 내용을 크롤링하여 모든 데이터를 news_data_detailed.csv에 저장합니다.

In [44]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time


# WebDriver 초기화 및 페이지 열기
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

output_data = []
fields = ["뉴스 제목", "뉴스 내용", "뉴스 반응 수", "신문사 이름", "기자 이름", "기자의 구독자 수", "기자의 응원해요 수"]
cnt = 0 # 크롤링 성공 카운트

for link in test_links:
    try:
        # 페이지 열기
        driver.get(link)

        # JavaScript 로드 대기
        time.sleep(2)
        
        # 주요 기사 정보 가져오기
        title = fetch_element_text(driver, "#title_area", "뉴스 제목을 가져오는 데 실패했습니다.")
        content = fetch_element_text(driver, '#dic_area', "뉴스 내용을 가져오는 데 실패했습니다.")
        news_like_count = fetch_element_text(driver, '.u_likeit_text', "뉴스 반응 수 정보를 가져오는 데 실패했습니다.")
        newspaper_name = fetch_element_attribute(driver, '.media_end_head_top_logo_img', 'alt', "신문사 이름을 가져오는 데 실패했습니다.")
        reporter_name = fetch_element_text(driver, '.media_journalistcard_summary_name_text', "기자 이름을 가져오는 데 실패했습니다.")
        reporter_subscribe_count = fetch_element_text(driver, '.media_journalistcard_summary_subscribe_value._SUBSCRIBE_COUNT_TEXT', "구독자 수 정보를 가져오는 데 실패했습니다.")
        reporter_like_count = fetch_element_text(driver, '.u_cnt', "기자의 응원해요 수 정보를 가져오는 데 실패했습니다.")

        # 기사 데이터 저장
        article_data = [
            title, content, news_like_count, newspaper_name,
            reporter_name, reporter_subscribe_count, reporter_like_count
        ]
        output_data.append(article_data)

        # 성공적으로 크롤링한 경우 카운트 증가 및 로그 출력
        cnt += 1
        # 완료 메시지
        print(f"({cnt}) 크롤링 완료: {link}")

    
    except Exception as e:
        print(f"URL 처리 중 오류 발생 : {e}")
        continue
    

# 브라우저 종료
driver.quit()
    
# CSV 파일로 저장
output_file = "news_data_detailed.csv"
save_to_csv(output_file, output_data, fields)

print(f"CSV 파일 '{output_file}'에 저장 완료!")

(1) 크롤링 완료: https://n.news.naver.com/mnews/article/417/0001053016
(2) 크롤링 완료: https://n.news.naver.com/mnews/article/009/0005431276
(3) 크롤링 완료: https://n.news.naver.com/mnews/article/018/0005927870
(4) 크롤링 완료: https://n.news.naver.com/mnews/article/030/0003277272
(5) 크롤링 완료: https://n.news.naver.com/mnews/article/014/0005297648
(6) 크롤링 완료: https://n.news.naver.com/mnews/article/008/0005143132
(7) 크롤링 완료: https://n.news.naver.com/mnews/article/008/0005143145
(8) 크롤링 완료: https://n.news.naver.com/mnews/article/009/0005431278
(9) 크롤링 완료: https://n.news.naver.com/mnews/article/028/0002727383
(10) 크롤링 완료: https://n.news.naver.com/mnews/article/009/0005431250
(11) 크롤링 완료: https://n.news.naver.com/mnews/article/366/0001048268
(12) 크롤링 완료: https://n.news.naver.com/mnews/article/014/0005297614
(13) 크롤링 완료: https://n.news.naver.com/mnews/article/421/0008030441
(14) 크롤링 완료: https://n.news.naver.com/mnews/article/003/0013024040
(15) 크롤링 완료: https://n.news.naver.com/mnews/article/003/0013024041
(16)